# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [ ]:
# Import modules
import numpy as np
from types import SimpleNamespace
from scipy import optimize 
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# autoreload modules when code is run. Otherwise, python will not see recent changes.
%load_ext autoreload
%autoreload 2

# Import your own code
from inauguralproject import ExchangeEconomyClass

# Question 1

I use the utility functions defined in the py-file, and set up a for-loop with conditions for pareto improvements of the initial endowment.

In [ ]:
# Call the model from the py-file
from inauguralproject import ExchangeEconomyClass
model = ExchangeEconomyClass()
par = model.par

# Create discrete list
N = 75
list_1 = [i/N for i in range(N+1)]

# Create empty lists for the results
list_x1A = []
list_x2A = []

# Set up conditions for pareto improvements and append the pareto approvements to the empty lists
for x1A in list_1:
    for x2A in list_1:
        x1B = 1 - x1A
        x2B = 1 - x2A
        if model.utility_A(x1A,x2A)>=model.utility_A(par.w1A,par.w2A) and model.utility_B(x1B,x2B)>=model.utility_B(par.w1B,par.w2B):
            list_x1A.append(x1A)
            list_x2A.append(x2A)

print("Pareto improvements are:", list_x1A , list_x2A)

The for-loop creates a list of pareto improvements. I plot the pareto improvement below with blue circles. The initial endowment is shown by the black square.

In [ ]:
par = model.par

# a. total endowment
w1bar = 1.0
w2bar = 1.0

# b. figure set up
fig = plt.figure(frameon=False,figsize=(6,6), dpi=100)
ax_A = fig.add_subplot(1, 1, 1)

ax_A.set_xlabel("$x_1^A$")
ax_A.set_ylabel("$x_2^A$")

temp = ax_A.twinx()
temp.set_ylabel("$x_2^B$")
ax_B = temp.twiny()
ax_B.set_xlabel("$x_1^B$")
ax_B.invert_xaxis()
ax_B.invert_yaxis()

# A
ax_A.scatter(par.w1A,par.w2A,marker='s',color='black',label='initial endowment')

# B
ax_A.scatter(list_x1A,list_x2A,marker='o',color='blue',label='pareto-improvement')


# limits
ax_A.plot([0,w1bar],[0,0],lw=2,color='black')
ax_A.plot([0,w1bar],[w2bar,w2bar],lw=2,color='black')
ax_A.plot([0,0],[0,w2bar],lw=2,color='black')
ax_A.plot([w1bar,w1bar],[0,w2bar],lw=2,color='black')

ax_A.set_xlim([-0.1, w1bar + 0.1])
ax_A.set_ylim([-0.1, w2bar + 0.1])    
ax_B.set_xlim([w1bar + 0.1, -0.1])
ax_B.set_ylim([w2bar + 0.1, -0.1])

ax_A.legend(frameon=True,loc='upper right',bbox_to_anchor=(1.8,1.0));

# Question 2

I create a list that p1 should be part of and use a for-loop on the check_market_clearing function from the py-file to print the market errors for given prices of p1.

In [ ]:
# Create discrete list for p1
N = 75
p_1 = [0.5+2*i/N for i in range(N+1)]


# Call the model from the py-file
model = ExchangeEconomyClass()
model.check_market_clearing


# Solve model for errors
for p1 in p_1:
    error = model.check_market_clearing(p1)
    print(p1, error)


From the print, it is clear that there is a root where eps1 changes from a positive number to a negtive number, and eps2 changes from a negative number to a positive number. The market clearing price for p1 is then between 0,9267 and 0,9534

# Question 3

I use the optimizer to find the market clearing price, by searching for the root of eps1. 

In [ ]:

# Define an objective function (to find root)
def obj(p1):
    return model.check_market_clearing(p1)[0]

# The function optimize.root will solve objective = 0 starting from x0 = 1, which is the guess of the price of p1
res = optimize.root(fun = obj, x0 = 1)

# Print the 
print('The market clearing price: ' , res.x)

The market clearing price is p1 = 0.944 when p2 = 1 (set as numeraire).
The walras law states that market clearing in n-1 markets also insures market clearing in the last market. 

# Question 4a

In [ ]:
# Call the model from the py-file
from inauguralproject import ExchangeEconomyClass
model = ExchangeEconomyClass()
par = model.par

# define objective function to minimize the negative utility function to find maximum utility for consumer A
def max_utility_A(p1):
    objective = lambda x1B, x2B: -model.utility_A(1 - x1B, 1 - x2B)  
    result = minimize(objective, bounds=(0, 1), method='bounded', x0=0)
    xA = result.x
    xB = 1 - xA
    return p1, xA, xB

# Find allocation for given price range
allocations_p_1 = [max_utility_A(p1) for p1 in p_1]
print("Allocations with prices in P1:", allocations_p_1)


In [ ]:
# Call the model from the py-file
from inauguralproject import ExchangeEconomyClass
model = ExchangeEconomyClass()
par = model.par

# Define objective function to maximize Consumer A's utility
def objective(x):
    return -model.utility_A(x[0],x[1])

# Initial guess for allocation
initial_x = [0 , 0]

# Bounds for allocation
bounds = ((0, 1) , (0, 1))

# Solve the optimization problem
result = minimize(objective, initial_x, bounds=bounds, method='SLSQP')

# Optimal allocation
optimal_allocation = result.x
optimal_prices_str = ', '.join([f'{p:.2f}' for p in optimal_prices])

print("Optimal prices:", optimal_prices_str)


# Question 4b

In [ ]:
# Call the model from the py-file
from inauguralproject import ExchangeEconomyClass
model = ExchangeEconomyClass()
par = model.par

# Define objective function to maximize Consumer A's utility
def objective(x):
    return -model.utility_A(x[0],x[1])

# Initial guess for allocation
initial_x = [0 , 0]

# Bounds for allocation
bounds = ((0, 1) , (0, 1))

# Solve the optimization problem
result = minimize(objective, initial_x, bounds=bounds, method='SLSQP')

# Optimal allocation
optimal_allocation = result.x


# Question 5a

# Question 5b

In [ ]:
from inauguralproject import ExchangeEconomyClass
model = ExchangeEconomyClass()
par = model.par

def objective(x):
    x1A, x2A = x
    return - model.utility_A(x)

def cons(x):
    x1B , x2B = x
    w1B , w2B = w
    return model.utility_B(x) - model.utility_B(w)

cons = {}

# Bounds for x
bounds = ()


# Solve the optimization problem
result = minimize(objective, initial_prices, bounds=bounds, method='SLSQP', constraints={'type': 'eq', 'fun': lambda p1: -model.check_market_clearing(p1)})


# Question 6a

In [ ]:
from inauguralproject import ExchangeEconomyClass
model = ExchangeEconomyClass()
par = model.par

def max_utility(x1A, x2A, x1B, x2B):
    return - model.utility_A(x1A, x2A)-model.utility_B(x1B, x2B)

# Initial guess for allocation
initial_x = [0 , 0 , 0 ,0]

# Bounds for allocation
bounds = ((0, 1) , (0, 1) , (0,1) , (0,1))

# Solve the optimization problem
result = minimize(max_utility, initial_x, bounds=bounds, method='SLSQP')


# Question 6b

# Question 7

In [ ]:
np.random.seed(2002)

# Defining the function W with the two elements with a uniform distribution 
def generate_w():
    w_1_A = np.random.uniform(0 , 1) 
    w_2_A = np.random.uniform(0 , 1)  
    return (w_1_A, w_2_A)

# Draw a set with 50 elements from W
for w_1_A in range(50):
    for w_2_B in range(50):
        W = list(generate_w())
        print(W)


# Question 8

In [ ]:
for w_1_A in list_1:

The project shows the Walras exchange economy. 